In [5]:
%matplotlib
import numpy as np
import scipy as sp
from scipy import linalg as la
import matplotlib.pyplot as plt
from glob import glob

Using matplotlib backend: TkAgg


In [81]:
#from tvregdiff import TVRegDiff as tvdiff

In [9]:
fis = glob('PWM*.csv')
noload = [fi for fi in fis if 'L' not in fi]
load = [fi for fi in fis if 'L' in fi]

In [25]:
cvt = {0: lambda s : float(s[1:]), 2: lambda s : float(s[:-1])}
def data(fis):
    d = []
    for fi in fis:
        d.append(np.genfromtxt(fi,skip_header=1,delimiter=',',converters=cvt))
    return np.asarray(d)

nd = data(noload)
ld = data(load)

In [80]:
t,q,dq = nd[0,:,0],nd[...,1].T,nd[...,2].T

q = np.unwrap(q,axis=0)
q_ = t[:,np.newaxis]*(np.diff(q,axis=0).T/np.diff(t))[-100:].mean(axis=1)
r = q - q_
dq_ = np.mean(dq,axis=0)
dr = (dq - dq_)/1000

n = 100

fig = plt.figure(1); plt.clf();
ax = plt.subplot(2,1,1); ax.grid('on')
ax.plot(t[n:],(r - r[-100:].mean(axis=0))[n:])
ax.set_ylabel('angle residual (rad)')
ax = plt.subplot(2,1,2); ax.grid('on')
#ax.plot(np.diff(q,axis=0))
#ax.plot(dq/1000,'--')
ax.plot(t[n+1:],(np.diff(r,axis=0)/np.diff(t)[:,np.newaxis])[n:])
#ax.plot(t[n:],dr[n:],'--')
ax.set_ylabel('velocity residual (rad/msec)')

ax.set_xlabel('time (msec)')
fig.savefig('residuals.pdf')
fig.savefig('residuals.png')